In [3]:
#Dependencies
import numpy as np
import pandas as pd
#dataset import
dataset = pd.read_csv('Oxides_firstnumpy.csv') #You need to change #directory accordingly
dataset.head(10) #Return 10 rows of datab

,Atom1,Atom2,Atom3,Atom4,Perovskite_label
0,3,23,32,8,0
1,55,13,14,8,0
2,20,27,74,8,1
3,47,23,33,8,0
4,38,30,34,8,0
5,3,32,52,8,0
6,56,25,75,8,0
7,20,30,14,8,0
8,38,20,92,8,1
9,82,12,92,8,1


In [10]:
#Changing pandas dataframe to numpy array
X = dataset.iloc[:,:4].values
y = dataset.iloc[:,4:5].values

#Normalizing the data
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

max_abs_scaler = preprocessing.MaxAbsScaler()
X = max_abs_scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [26]:
input_shape = (4,)

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Activation
from tensorflow.python.keras.optimizer_v2 import rmsprop


def get_model(input_shape, dropout2_rate=0.5):
    """Builds a Sequential CNN model to recognize structure.

    Args:
      input_shape: Shape of the input depending on the `data_format`.
      dropout2_rate: float between 0 and 1. Fraction of the input units to drop for `dropout_2` layer.

    Returns:
      a Keras model

    """
    # Reset the tensorflow backend session.
    # tf.keras.backend.clear_session()
    # Define a CNN model to recognize MNIST.
    model = Sequential()
    
    model.add(Dense(100, activation='relu', input_shape=(4,)))

    model.add(Dropout(0.2))
    
    model.add(Dense(150, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, activation='sigmoid'))
    
    #model.add(Conv2D(32, kernel_size=(3, 3),
    #                 activation='relu',
    #                 input_shape=input_shape,
    #                 name="conv2d_1"))
    #model.add(Conv2D(64, (3, 3), activation='relu', name="conv2d_2"))
    #model.add(MaxPooling2D(pool_size=(2, 2), name="maxpool2d_1"))
    #model.add(Dropout(0.25, name="dropout_1"))
    #model.add(Flatten(name="flatten"))
    #model.add(Dense(128, activation='relu', name="dense_1"))
    #model.add(Dropout(dropout2_rate, name="dropout_2"))
    #model.add(Dense(NUM_CLASSES, activation='softmax', name="dense_2"))
    
    
    return model

In [34]:
def fit_with(input_shape, verbose, dropout2_rate, lr):

    # Create the model using a specified hyperparameters.
    model = get_model(input_shape, dropout2_rate)

    # Train the model for a specified number of epochs.
    optimizer = rmsprop.RMSProp(learning_rate=lr)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # Train the model with the train dataset.
    model.fit(x=X_train, epochs=1, steps_per_epoch=468,
              batch_size=64, verbose=verbose)

    # Evaluate the model with the eval dataset.
    score = model.evaluate(eval_ds, steps=10, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Return the accuracy.

    return score[1]

from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, input_shape, verbose)

In [35]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout2_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(n_iter=4)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------


ValueError: in user code:

    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:789 run_step

In [20]:
pbounds = {'dropout2_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2), "dense_1_neurons_x128": (0.9, 3.1)}

def fit_with(input_shape, verbose, dropout2_rate, dense_1_neurons_x128, lr):

    # Create the model using a specified hyperparameters.
    dense_1_neurons = max(int(dense_1_neurons_x128 * 128), 128)
    model = get_model(input_shape, dropout2_rate, dense_1_neurons)
    # ...

# ANOTHER TRY

In [40]:
import GPy, GPyOpt
import numpy as np
import pandas as pds
import random
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [56]:
# MNIST class
class MNIST():
    def __init__(self, first_input=4, last_output=1,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.__first_input = first_input
        self.__last_output = last_output
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        
                #Changing pandas dataframe to numpy array
        X = dataset.iloc[:,:4].values
        y = dataset.iloc[:,4:5].values
        
        #Normalizing the data
        from sklearn.preprocessing import StandardScaler
        from sklearn import preprocessing
        
        max_abs_scaler = preprocessing.MaxAbsScaler()
        X = max_abs_scaler.fit_transform(X)
        
        from sklearn.model_selection import train_test_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = train_test_split(X,y,test_size = 0.3)
        
        self.__model = self.mnist_model()
        

    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(self.__first_input,)))
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(self.__last_output))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def mnist_evaluate(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [57]:
# function to run mnist class
def run_mnist(first_input=4, last_output=1,
              l1_out=512, l2_out=512, 
              l1_drop=0.2, l2_drop=0.2, 
              batch_size=100, epochs=10, validation_split=0.1):
    
    _mnist = MNIST(first_input=first_input, last_output=last_output,
                   l1_out=l1_out, l2_out=l2_out, 
                   l1_drop=l1_drop, l2_drop=l2_drop, 
                   batch_size=batch_size, epochs=epochs, 
                   validation_split=validation_split)
    mnist_evaluation = _mnist.mnist_evaluate()
    return mnist_evaluation

In [68]:
# bounds for hyper-parameters in mnist model
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l2_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'l2_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (2, 5, 10)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (100, 500, 1000)}]

In [69]:
# function to optimize mnist model
def f(x):
    print(x)
    evaluation = run_mnist(
        l1_drop = float(x[:,1]), 
        l2_drop = float(x[:,2]), 
        l1_out = int(x[:,3]),
        l2_out = int(x[:,4]), 
        batch_size = int(x[:,5]), 
        epochs = int(x[:,6]), 
        validation_split = float(x[:,0]))
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    return evaluation[0]

In [70]:
# optimizer
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

[[4.03748137e-02 1.80748035e-01 1.06649679e-01 6.40000000e+01
  2.56000000e+02 5.00000000e+00 5.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.4035874307155609
[0.0, 0.4035874307155609]
[[2.80930623e-01 2.71502165e-01 1.20690039e-01 5.12000000e+02
  2.56000000e+02 5.00000000e+00 1.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.35426008701324463
[0.0, 0.35426008701324463]
[[2.61660584e-01 7.50074132e-02 1.17067734e-01 5.12000000e+02
  1.28000000e+02 1.00000000e+01 5.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.36771300435066223
[0.0, 0.36771300435066223]
[[1.88160837e-01 2.68914294e-01 1.82234977e-01 2.56000000e+02
  1.28000000e+02 5.00000000e+00 5.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.39461883902549744
[0.0, 0.39461883902549744]
[[6.62479519e-03 2.47160287e-01 2.38151378e-01 6.40000000e+01
  5.12000000e+02 5.00000000e+00 1.00000000e+03]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.4080717

In [71]:
# optimize mnist model
opt_mnist.run_optimization(max_iter=10)

[[1.59733386e-01 2.21930601e-01 1.62435694e-01 1.28000000e+02
  5.12000000e+02 2.00000000e+00 1.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.40807175636291504
[0.0, 0.40807175636291504]
[[2.28449776e-01 1.20495064e-02 1.70533150e-01 1.28000000e+02
  6.40000000e+01 5.00000000e+00 1.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.3766816258430481
[0.0, 0.3766816258430481]
[[1.04430386e-01 2.38181206e-01 2.74049915e-02 6.40000000e+01
  1.28000000e+02 5.00000000e+00 1.00000000e+03]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.36771300435066223
[0.0, 0.36771300435066223]
[[7.98325993e-02 1.06572577e-03 2.08807419e-01 2.56000000e+02
  1.02400000e+03 2.00000000e+00 1.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.3991031348705292
[0.0, 0.3991031348705292]
[[5.30036261e-02 4.48101138e-02 2.32493952e-01 6.40000000e+01
  6.40000000e+01 2.00000000e+00 1.00000000e+02]]
Epoch 00002: early stopping
LOSS:	0.0 	 ACCURACY:	0.385650217

In [72]:
# print optimized mnist model
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
\t{8}:\t{9}
\t{10}:\t{11}
\t{12}:\t{13}
""".format(bounds[0]["name"],opt_mnist.x_opt[0],
           bounds[1]["name"],opt_mnist.x_opt[1],
           bounds[2]["name"],opt_mnist.x_opt[2],
           bounds[3]["name"],opt_mnist.x_opt[3],
           bounds[4]["name"],opt_mnist.x_opt[4],
           bounds[5]["name"],opt_mnist.x_opt[5],
           bounds[6]["name"],opt_mnist.x_opt[6]))
print("optimized loss: {0}".format(opt_mnist.fx_opt))


Optimized Parameters:
	validation_split:	0.040374813715259514
	l1_drop:	0.1807480353942922
	l2_drop:	0.10664967944542196
	l1_out:	64.0
	l2_out:	256.0
	batch_size:	5.0
	epochs:	500.0

optimized loss: 0.0


In [73]:
opt_mnist.x_opt

array([4.03748137e-02, 1.80748035e-01, 1.06649679e-01, 6.40000000e+01,
       2.56000000e+02, 5.00000000e+00, 5.00000000e+02])

In [64]:
model.evaluate(X_test, y_test)[1]

NameError: name 'model' is not defined